# Summer Analytics 2025 – Final Project
## Pricing for Urban Parking Lots
 Name: Chirag Yadav
 Program: Summer Analytics, IIT Guwahati  
 Week: Final Project (Post Week 5)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1: Load Dataset and Explore

In [5]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/SummerAnalytics2025/dataset.csv')


##  Step 2: Model 1 – Linear Pricing


In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load dataset from Google Drive or local upload
# Please ensure the file path is correct and Google Drive is mounted
df = pd.read_csv('/content/drive/MyDrive/SummerAnalytics2025/dataset.csv')

# ----------------------------
# STEP 1: Preprocessing
# ----------------------------

# Convert LastUpdatedTime to 30-minute interval time steps
def get_time_step(t):
    base = datetime.strptime("08:00:00", "%H:%M:%S")
    current = datetime.strptime(t, "%H:%M:%S")
    return (current - base).seconds // 1800

df['TimeStep'] = df['LastUpdatedTime'].apply(get_time_step)

# Sort values for logical price progression
df = df.sort_values(by=['SystemCodeNumber', 'LastUpdatedDate', 'TimeStep']).reset_index(drop=True)

# ----------------------------
# STEP 2: Model 1 Parameters
# ----------------------------

BASE_PRICE = 10.0
ALPHA = 2.0  # price sensitivity to occupancy ratio

# ----------------------------
# STEP 3: Model 1 Pricing Logic
# ----------------------------

def apply_model_1(df, base_price=BASE_PRICE, alpha=ALPHA):
    df['Model1_Price'] = np.nan

    # Loop by ParkingLot and Date
    for (lot, date), group in df.groupby(['SystemCodeNumber', 'LastUpdatedDate']):
        prev_price = base_price
        for idx in group.index:
            occ = df.loc[idx, 'Occupancy']
            cap = df.loc[idx, 'Capacity']
            ratio = occ / cap if cap > 0 else 0  # avoid division by 0
            new_price = prev_price + alpha * ratio
            df.at[idx, 'Model1_Price'] = round(new_price, 2)
            prev_price = new_price  # update for next time step
    return df

# Apply the model
df = apply_model_1(df)

# ----------------------------
# STEP 4: Preview the Results
# ----------------------------
df[['SystemCodeNumber', 'LastUpdatedDate', 'TimeStep', 'Occupancy', 'Capacity', 'Model1_Price']].head(10)

,SystemCodeNumber,LastUpdatedDate,TimeStep,Occupancy,Capacity,Model1_Price
0,BHMBCCMKT01,01-11-2016,0,50,577,10.17
1,BHMBCCMKT01,01-11-2016,0,54,577,10.36
2,BHMBCCMKT01,01-11-2016,2,69,577,10.60
3,BHMBCCMKT01,01-11-2016,2,91,577,10.92
4,BHMBCCMKT01,01-11-2016,4,135,577,11.38
5,BHMBCCMKT01,01-11-2016,4,163,577,11.95
6,BHMBCCMKT01,01-11-2016,6,185,577,12.59
7,BHMBCCMKT01,01-11-2016,6,207,577,13.31
8,BHMBCCMKT01,01-11-2016,8,239,577,14.14
9,BHMBCCMKT01,01-11-2016,8,244,577,14.98


## Step 3: Model 2 – Demand-Based Pricing


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/SummerAnalytics2025/dataset.csv')

# Compute TimeStep from LastUpdatedTime
def get_time_step(t):
    base = datetime.strptime("08:00:00", "%H:%M:%S")
    current = datetime.strptime(t, "%H:%M:%S")
    return (current - base).seconds // 1800

df['TimeStep'] = df['LastUpdatedTime'].apply(get_time_step)

# Sort the dataset
df = df.sort_values(by=['SystemCodeNumber', 'LastUpdatedDate', 'TimeStep']).reset_index(drop=True)

# -------------------------------
# Define Model 2 Parameters
# -------------------------------
BASE_PRICE = 10.0
ALPHA = 2.0
BETA = 0.3
GAMMA = 0.5
DELTA = 1.0
EPSILON = 0.2
LAMBDA = 0.5

# Map vehicle types to weights, using a default value of 1.0 for unknown types
vehicle_weight_map = {'Car': 1.0, 'Bike': 0.5, 'Truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weight_map).fillna(1.0)


# Convert IsSpecialDay to binary (if not already)
df['IsSpecialDay'] = df['IsSpecialDay'].astype(int)

# -------------------------------
# Step 1: Compute Raw Demand Score
# -------------------------------
def compute_demand(row):
    occ_ratio = row['Occupancy'] / row['Capacity'] if row['Capacity'] > 0 else 0
    return (
        ALPHA * occ_ratio +
        BETA * row['QueueLength'] -
        GAMMA * (1 if row['TrafficConditionNearby'] == 'high' else (0.5 if row['TrafficConditionNearby'] == 'average' else 0)) + # Map traffic condition to numerical value
        DELTA * row['IsSpecialDay'] +
        EPSILON * row['VehicleWeight']
    )

df['RawDemand'] = df.apply(compute_demand, axis=1)

# -------------------------------
# Step 2: Normalize Demand (per lot per day)
# -------------------------------
df['NormalizedDemand'] = df.groupby(['SystemCodeNumber', 'LastUpdatedDate'])['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
)

# -------------------------------
# Step 3: Compute Final Price
# -------------------------------
df['Model2_Price'] = df['NormalizedDemand'].apply(lambda d: round(BASE_PRICE * (1 + LAMBDA * d), 2))

# -------------------------------
# Preview Final Dataset
# -------------------------------
df[['SystemCodeNumber', 'LastUpdatedDate', 'TimeStep', 'Occupancy', 'QueueLength', 'TrafficConditionNearby', 'VehicleType', 'RawDemand', 'NormalizedDemand', 'Model2_Price']].head(10)

,SystemCodeNumber,LastUpdatedDate,TimeStep,Occupancy,QueueLength,TrafficConditionNearby,VehicleType,RawDemand,NormalizedDemand,Model2_Price
0,BHMBCCMKT01,01-11-2016,0,50,1,low,car,0.673310,0.000000,10.00
1,BHMBCCMKT01,01-11-2016,0,54,1,low,car,0.687175,0.006111,10.03
2,BHMBCCMKT01,01-11-2016,2,69,2,average,car,0.789168,0.051062,10.26
3,BHMBCCMKT01,01-11-2016,2,91,1,low,car,0.815425,0.062634,10.31
4,BHMBCCMKT01,01-11-2016,4,135,3,average,truck,1.317938,0.284105,11.42
5,BHMBCCMKT01,01-11-2016,4,163,4,average,truck,1.714991,0.459097,12.30
6,BHMBCCMKT01,01-11-2016,6,185,5,average,car,2.091248,0.624924,13.12
7,BHMBCCMKT01,01-11-2016,6,207,4,average,bike,1.867504,0.526314,12.63
8,BHMBCCMKT01,01-11-2016,8,239,7,high,car,2.628423,0.861671,14.31
9,BHMBCCMKT01,01-11-2016,8,244,7,high,car,2.645754,0.869310,14.35


## Step 4: Model 3 – Competitive Pricing


In [12]:
from math import sqrt

# Thresholds
DIST_THRESHOLD = 0.02  # ~2km
ETA = 0.3              # Competitive influence on pricing

# Step 1: Create a map of ParkingLot locations
location_map = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first()

# Step 2: Find nearby competitors for each lot
def get_nearby_lots(lot_id):
    lat1, lon1 = location_map.loc[lot_id]
    nearby = []
    for other_id in location_map.index:
        if other_id == lot_id:
            continue
        lat2, lon2 = location_map.loc[other_id]
        dist = sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)
        if dist <= DIST_THRESHOLD:
            nearby.append(other_id)
    return nearby

# Build the neighbor map
nearby_lots_map = {lot: get_nearby_lots(lot) for lot in location_map.index}

# Step 3: Compute Model 3 with rerouting logic
def compute_model3_price_with_reroute(df):
    df['Model3_Price'] = df['Model2_Price']  # Start from Model 2
    df['SuggestedRerouteTo'] = np.nan        # New column for rerouting

    for idx, row in df.iterrows():
        lot = row['SystemCodeNumber']
        date = row['LastUpdatedDate']
        timestep = row['TimeStep']
        curr_price = row['Model2_Price']
        occ = row['Occupancy']
        cap = row['Capacity']

        # Skip if no neighbors
        neighbors = nearby_lots_map.get(lot, [])
        if not neighbors:
            continue

        # Get neighbors’ data at same time step
        mask = (df['SystemCodeNumber'].isin(neighbors)) & \
               (df['LastUpdatedDate'] == date) & \
               (df['TimeStep'] == timestep)

        neighbor_df = df.loc[mask, ['SystemCodeNumber', 'Occupancy', 'Capacity', 'Model2_Price']]

        if neighbor_df.empty:
            continue

        # Compute average neighbor price
        avg_neighbor_price = neighbor_df['Model2_Price'].mean()

        # Default: no change
        adjusted_price = curr_price
        suggested_reroute = np.nan

        # Check for reroute condition: lot full AND cheaper neighbor
        if cap > 0 and occ / cap >= 0.9:
            cheaper_neighbors = neighbor_df[neighbor_df['Model2_Price'] < curr_price]
            if not cheaper_neighbors.empty:
                # Reroute to the cheapest available nearby lot
                suggested_reroute = cheaper_neighbors.sort_values('Model2_Price').iloc[0]['SystemCodeNumber']
                # Also reduce your own price slightly
                adjusted_price = curr_price - ETA * (curr_price - avg_neighbor_price)

        # Else: increase price if others are expensive and you're not full
        elif curr_price < avg_neighbor_price:
            adjusted_price = curr_price + ETA * (avg_neighbor_price - curr_price)

        # Save outputs
        df.at[idx, 'Model3_Price'] = round(adjusted_price, 2)
        df.at[idx, 'SuggestedRerouteTo'] = suggested_reroute

    return df

# Apply enhanced Model 3
df = compute_model3_price_with_reroute(df)

# Preview results
df[['SystemCodeNumber', 'LastUpdatedDate', 'TimeStep', 'Model2_Price', 'Model3_Price', 'SuggestedRerouteTo']].head(10)


/tmp/ipython-input-12-2423680535.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Others-CCCPS119a' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'SuggestedRerouteTo'] = suggested_reroute


,SystemCodeNumber,LastUpdatedDate,TimeStep,Model2_Price,Model3_Price,SuggestedRerouteTo
0,BHMBCCMKT01,01-11-2016,0,10.00,10.09,NaN
1,BHMBCCMKT01,01-11-2016,0,10.03,10.11,NaN
2,BHMBCCMKT01,01-11-2016,2,10.26,10.51,NaN
3,BHMBCCMKT01,01-11-2016,2,10.31,10.55,NaN
4,BHMBCCMKT01,01-11-2016,4,11.42,11.67,NaN
5,BHMBCCMKT01,01-11-2016,4,12.30,12.30,NaN
6,BHMBCCMKT01,01-11-2016,6,13.12,13.26,NaN
7,BHMBCCMKT01,01-11-2016,6,12.63,12.92,NaN
8,BHMBCCMKT01,01-11-2016,8,14.31,14.45,NaN
9,BHMBCCMKT01,01-11-2016,8,14.35,14.48,NaN


## Step 5: Real-Time Streaming Integration with Pathway


## Step 6: Bokeh Visualization


## Final: Summary, Insights, and Report Content
